# Data Cleaning
We first deal with the data we scraped from google map.(credit by https://apify.com/)


In [1]:
import json

file_path = "data/dataset_crawler-google-places_2024-12-16_02-57-12-340.json"

with open(file_path, 'r') as f:
    data = json.load(f)

print(type(data)) 
print(len(data))   
print(data[0])     


<class 'list'>
180
{'searchString': 'Michelin restaurants NYC', 'rank': 31, 'searchPageUrl': 'https://www.google.com/maps/search/Michelin%20restaurants%20NYC/@40.619187165763144,-74.04782588816352,15z?hl=en', 'searchPageLoadedUrl': None, 'isAdvertisement': False, 'title': 'Phil-Am Kusina', 'subTitle': None, 'description': 'Casual restaurant cooking up traditional Filipino dishes with a modern twist.', 'price': '$10–20', 'categoryName': 'Filipino restaurant', 'address': '556 Tompkins Ave., Staten Island, NY 10305', 'neighborhood': 'Rosebank', 'street': '556 Tompkins Ave.', 'city': 'Staten Island', 'postalCode': '10305', 'state': 'New York', 'countryCode': 'US', 'website': 'http://www.philamkusina.com/', 'phone': '(718) 727-3663', 'phoneUnformatted': '+17187273663', 'claimThisBusiness': False, 'location': {'lat': 40.6124623, 'lng': -74.071427}, 'locatedIn': None, 'plusCode': 'JW6H+XC Staten Island, New York', 'menu': None, 'totalScore': 4.7, 'permanentlyClosed': False, 'temporarilyClosed

## Convert JSON data to a DataFrame

In [2]:
import pandas as pd

df = pd.DataFrame(data)

print(df.head())

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
                                 searchString  rank  \
0                    Michelin restaurants NYC    31   
1  Michelin-starred restaurants New York City    32   
2  Michelin-starred restaurants New York City    29   
3                    Michelin restaurants NYC    88   
4                    Michelin restaurants NYC    86   

                                       searchPageUrl searchPageLoadedUrl  \
0  https://www.google.com/maps/search/Michelin%20...                None   
1  https://www.go

In [3]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 10)      # Limit rows to avoid long output

# View the DataFrame
print(df.head())

                                 searchString  rank  \
0                    Michelin restaurants NYC    31   
1  Michelin-starred restaurants New York City    32   
2  Michelin-starred restaurants New York City    29   
3                    Michelin restaurants NYC    88   
4                    Michelin restaurants NYC    86   

                                       searchPageUrl searchPageLoadedUrl  \
0  https://www.google.com/maps/search/Michelin%20...                None   
1  https://www.google.com/maps/search/Michelin-st...                None   
2  https://www.google.com/maps/search/Michelin-st...                None   
3  https://www.google.com/maps/search/Michelin%20...                None   
4  https://www.google.com/maps/search/Michelin%20...                None   

   isAdvertisement                     title subTitle  \
0            False            Phil-Am Kusina     None   
1            False              H & L Bagels     None   
2            False        Coszcal De Alle

## Select the columns we need

In [4]:
# List of desired columns
desired_columns = [
    "title", "price", "categoryName", "address", "neighborhood",
    "street", "city", "postalCode", "state", "location",
    "totalScore", "reviewsCount", "url", "reviews"
]

# Filter the DataFrame to keep only the desired columns
df_selected = df[desired_columns]

# Display the resulting DataFrame
print(df_selected.head())


                      title   price         categoryName  \
0            Phil-Am Kusina  $10–20  Filipino restaurant   
1              H & L Bagels   $1–10           Bagel shop   
2        Coszcal De Allende  $30–50   Mexican restaurant   
3                    E.A.T.  $30–50  American restaurant   
4  Dowling's at The Carlyle   $100+           Restaurant   

                                      address   neighborhood  \
0  556 Tompkins Ave., Staten Island, NY 10305       Rosebank   
1            8818 3rd Ave, Brooklyn, NY 11209  Fort Hamilton   
2            7506 3rd Ave, Brooklyn, NY 11209      Bay Ridge   
3        1064 Madison Ave, New York, NY 10028      Manhattan   
4            35 E 76th St, New York, NY 10021      Manhattan   

              street           city postalCode     state  \
0  556 Tompkins Ave.  Staten Island      10305  New York   
1       8818 3rd Ave       Brooklyn      11209  New York   
2       7506 3rd Ave       Brooklyn      11209  New York   
3   1064 Madis

## Refine the review column

In [5]:
# Explode the 'reviews' column to split lists into rows
df_exploded = df_selected.explode('reviews', ignore_index=True)

# Extract 'text' from the dictionary in each exploded row
df_exploded['review_text'] = df_exploded['reviews'].apply(
    lambda x: x.get('text') if isinstance(x, dict) else None
)

# Drop the original 'reviews' column (optional)
df_final = df_exploded.drop(columns=['reviews'])

# Filter out rows where 'review_text' is None
df_final = df_final.dropna(subset=['review_text'])

print(df_final)


                title   price         categoryName  \
1      Phil-Am Kusina  $10–20  Filipino restaurant   
2      Phil-Am Kusina  $10–20  Filipino restaurant   
3      Phil-Am Kusina  $10–20  Filipino restaurant   
4      Phil-Am Kusina  $10–20  Filipino restaurant   
5      Phil-Am Kusina  $10–20  Filipino restaurant   
...               ...     ...                  ...   
17342         Mao Mao  $20–30      Thai restaurant   
17344         Mao Mao  $20–30      Thai restaurant   
17347         Mao Mao  $20–30      Thai restaurant   
17349         Mao Mao  $20–30      Thai restaurant   
17350         Mao Mao  $20–30      Thai restaurant   

                                          address        neighborhood  \
1      556 Tompkins Ave., Staten Island, NY 10305            Rosebank   
2      556 Tompkins Ave., Staten Island, NY 10305            Rosebank   
3      556 Tompkins Ave., Staten Island, NY 10305            Rosebank   
4      556 Tompkins Ave., Staten Island, NY 10305          

## Convert location column into geometry

In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

df_selected['latitude'] = df_selected['location'].apply(lambda x: x.get('lat') if isinstance(x, dict) else None)
df_selected['longitude'] = df_selected['location'].apply(lambda x: x.get('lng') if isinstance(x, dict) else None)

geometry = [Point(lon, lat) for lon, lat in zip(df_selected['longitude'], df_selected['latitude'])]

gdf = gpd.GeoDataFrame(df_selected, geometry=geometry, crs="EPSG:4326")  # WGS84 Coordinate System

print(gdf.head())




                      title   price         categoryName  \
0            Phil-Am Kusina  $10–20  Filipino restaurant   
1              H & L Bagels   $1–10           Bagel shop   
2        Coszcal De Allende  $30–50   Mexican restaurant   
3                    E.A.T.  $30–50  American restaurant   
4  Dowling's at The Carlyle   $100+           Restaurant   

                                      address   neighborhood  \
0  556 Tompkins Ave., Staten Island, NY 10305       Rosebank   
1            8818 3rd Ave, Brooklyn, NY 11209  Fort Hamilton   
2            7506 3rd Ave, Brooklyn, NY 11209      Bay Ridge   
3        1064 Madison Ave, New York, NY 10028      Manhattan   
4            35 E 76th St, New York, NY 10021      Manhattan   

              street           city postalCode     state  \
0  556 Tompkins Ave.  Staten Island      10305  New York   
1       8818 3rd Ave       Brooklyn      11209  New York   
2       7506 3rd Ave       Brooklyn      11209  New York   
3   1064 Madis

/var/folders/q7/ww3mcfxj1pv1181n2srlnm780000gn/T/ipykernel_27933/3357681030.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['latitude'] = df_selected['location'].apply(lambda x: x.get('lat') if isinstance(x, dict) else None)
/var/folders/q7/ww3mcfxj1pv1181n2srlnm780000gn/T/ipykernel_27933/3357681030.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['longitude'] = df_selected['location'].apply(lambda x: x.get('lng') if isinstance(x, dict) else None)


In [7]:
df_final['latitude'] = df_final['location'].apply(lambda x: x.get('lat') if isinstance(x, dict) else None)
df_final['longitude'] = df_final['location'].apply(lambda x: x.get('lng') if isinstance(x, dict) else None)

geometry = [Point(lon, lat) for lon, lat in zip(df_final['longitude'], df_final['latitude'])]

df_final = gpd.GeoDataFrame(df_final, geometry=geometry, crs="EPSG:4326")

In [8]:
output_path = "/Users/luoxiaoyi/Downloads/24fall-python-final-proposal-kuma_chloe_wenjun_finalproposal-main/pythonfinal/data/restaurants_reviews_final.geojson"
df_final.to_file(output_path, driver="GeoJSON")
print(f"File saved to: {output_path}")

File saved to: /Users/luoxiaoyi/Downloads/24fall-python-final-proposal-kuma_chloe_wenjun_finalproposal-main/pythonfinal/data/restaurants_reviews_final.geojson


## Interactive Map of Top-Rated Restaurants in New York City
This interactive map showcases a curated selection of restaurants across New York City, displaying key details such as name, address, and overall rating. Each marker represents a restaurant and provides additional information, including its location and review scores, to help explore dining options throughout the city.

In [9]:
import folium
from shapely.geometry import Point

map_center = [gdf['latitude'].mean(), gdf['longitude'].mean()] 
m = folium.Map(location=map_center, zoom_start=12)

for _, row in gdf.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(f"<b>{row['title']}</b><br>{row['address']}<br>Score: {row['totalScore']}", max_width=300),
        tooltip=row['title']
    ).add_to(m)

m.save("interactive_restaurants_map.html")
m


In [12]:
output_path = "/Users/luoxiaoyi/Downloads/24fall-python-final-proposal-kuma_chloe_wenjun_finalproposal-main/pythonfinal/data/restaurants_info.geojson"
gdf.to_file(output_path, driver="GeoJSON")

print(f"File saved to: {output_path}")

File saved to: /Users/luoxiaoyi/Downloads/24fall-python-final-proposal-kuma_chloe_wenjun_finalproposal-main/pythonfinal/data/restaurants_info.geojson


In [15]:
print("Unique values in 'price':", df_selected['price'].unique())


Unique values in 'price': ['$10–20' '$1–10' '$30–50' '$100+' '$20–30' '$50–100' '$$$' '$$' None
 '$$$$' '$']
